In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [2]:
# Load the keywords from a CSV file into a DataFrame
df = pd.read_csv('content/english_keywords.csv')

In [3]:
print(df.head(5))

                         Keyword Suggested By  Notes
0  gender equality law sri lanka      Bhushan     29
1   women's economic empowerment        Peter     28
2     gender inclusion sri lanka      Bhushan     26
3     social inclusion sri lanka      Bhushan     26
4       Discrimination Awareness        Hamza     24


In [8]:
# Filter the first 5 rows
#filtered_df = df.head(10).copy()

# Reset index for the new DataFrame
#filtered_df = filtered_df.reset_index(drop=True)
filtered_df=df

In [9]:
print(filtered_df)

                          Keyword Suggested By  Notes
0   gender equality law sri lanka      Bhushan     29
1    women's economic empowerment        Peter     28
2      gender inclusion sri lanka      Bhushan     26
3      social inclusion sri lanka      Bhushan     26
4        Discrimination Awareness        Hamza     24
..                            ...          ...    ...
79                          women        Peter      5
80                           PACT      Thilina      4
81                           UNDP      Thilina      4
82                           jobs        hamza      4
83                            NGO      Thilina      3

[84 rows x 3 columns]


In [10]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
url = 'https://www.adaderana.lk/search_results.php'


In [11]:
# Load the keywords from a CSV file into a DataFrame
#keywords_df = pd.read_csv('english_keywords.csv')

# Initialize a list to store dictionaries
result_data = []

# Iterate through each keyword in the DataFrame
for index, row in filtered_df.iterrows():
    keyword = row[0]  # Assuming 'keyword' is the column name in your CSV file
    payload = {'mode':'1','show':'1','query':keyword}
    r = requests.get(url, params=payload, headers=headers)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, "html.parser")
        # Select all <a> elements within <dt> elements with the class 'result-title'
        result_title_links = soup.select('a.newsItemTitle')
        # Extract href attributes and anchor text, and store them in the DataFrame
        for link in result_title_links:
            href = link.get('href')
            anchor_text = link.text.strip()
            result_data.append({'Keyword': keyword, 'Anchor Text': anchor_text, 'Href': href})

    time.sleep(random.uniform(1, 2))

# Create the DataFrame from the list of dictionaries
result_df = pd.DataFrame(result_data)

# Print or further process the resulting DataFrame
print(result_df.head())

                   Keyword                                        Anchor Text  \
0  female entrepreneurship         Sri Lanka female entrepreneurship in focus   
1  female entrepreneurship         Sri Lanka female entrepreneurship in focus   
2      workplace diversity                 Google fires diversity memo author   
3      workplace diversity                 Google fires diversity memo author   
4          decision-making  President vows to increase women participation...   

                 Href  
0  news.php?nid=29544  
1  news.php?nid=29544  
2  news.php?nid=42422  
3  news.php?nid=42422  
4  news.php?nid=88904  


In [12]:
result_df.count()

Keyword        14052
Anchor Text    14052
Href           14052
dtype: int64

In [13]:
unique_urls = result_df.drop_duplicates(subset=['Href'])
unique_urls.count()

Keyword        6412
Anchor Text    6412
Href           6412
dtype: int64

In [26]:
urls = pd.DataFrame()
urls['url']='https://www.adaderana.lk/' + unique_urls['Href']

In [27]:
# Filter the first 5 rows
urls_subset = urls.head(500).copy()

# Reset index for the new DataFrame
urls_subset= urls_subset.reset_index(drop=True)

urls = urls_subset

In [28]:
print(urls)

                                             url
0    https://www.adaderana.lk/news.php?nid=29544
1    https://www.adaderana.lk/news.php?nid=42422
2    https://www.adaderana.lk/news.php?nid=88904
3    https://www.adaderana.lk/news.php?nid=86306
4    https://www.adaderana.lk/news.php?nid=69459
..                                           ...
495  https://www.adaderana.lk/news.php?nid=18508
496  https://www.adaderana.lk/news.php?nid=16702
497  https://www.adaderana.lk/news.php?nid=16445
498  https://www.adaderana.lk/news.php?nid=14892
499  https://www.adaderana.lk/news.php?nid=14272

[500 rows x 1 columns]


In [29]:
article_data = []
for index,row in urls.iterrows():
    url = row['url']
    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, "html.parser")
            #set the html tag where newe content is present
            target_tag = soup.find_all('div', 'news-content')
            # Check if the target tag is found
            target=target_tag[0]
            if target:
              # Print the articleBody
              article_body = target.text
              time_tag = soup.find_all('p','news-datestamp')
              date_time_raw = soup.find('p', class_='news-datestamp').get_text(strip=True)
                # Split the string into date and time
              date, time = date_time_raw.split('\xa0\xa0')  # '\xa0' represents a non-breaking space
              #print(article_body)
              article_data.append({'URL': url, 'Title': soup.title.string, 'Text': article_body, 'Date':date, 'Time':time })
              print(url)
            else:
              print("target tag not found.")
        else:
              print(f"Failed to retrieve content. Status Code: {random.status_code}")
              time.sleep(random.uniform(1, 2))
    except requests.RequestException as e:
        print(f"Failed to retrieve content for {url}. Exception: {e}")
        # You may want to log the exception or handle it differently
        time.sleep(random.uniform(1, 2))
        continue  # Continue to the next iteration of the loop

    except Exception as e:
        print(f"An unexpected error occurred for {url}. Exception: {e}")
        # Handle the unexpected exception
        continue  # Continue to the next iteration of the loop

# Create the DataFrame from the list of dictionaries
result_df = pd.DataFrame(article_data)

# Print or further process the resulting DataFrame
print(result_df.head())

result_df.to_csv('content/results.csv', index=False,encoding='utf-8')

https://www.adaderana.lk/news.php?nid=29544
https://www.adaderana.lk/news.php?nid=42422
https://www.adaderana.lk/news.php?nid=88904
https://www.adaderana.lk/news.php?nid=86306
https://www.adaderana.lk/news.php?nid=69459
https://www.adaderana.lk/news.php?nid=46580
https://www.adaderana.lk/news.php?nid=34169
https://www.adaderana.lk/news.php?nid=26777
https://www.adaderana.lk/news.php?nid=88583
https://www.adaderana.lk/news.php?nid=79023
https://www.adaderana.lk/news.php?nid=73038
https://www.adaderana.lk/news.php?nid=72939
https://www.adaderana.lk/news.php?nid=72027
https://www.adaderana.lk/news.php?nid=71732
https://www.adaderana.lk/news.php?nid=59512
https://www.adaderana.lk/news.php?nid=53213
https://www.adaderana.lk/news.php?nid=50369
https://www.adaderana.lk/news.php?nid=49637
https://www.adaderana.lk/news.php?nid=44287
https://www.adaderana.lk/news.php?nid=43950
https://www.adaderana.lk/news.php?nid=43897
https://www.adaderana.lk/news.php?nid=43315
https://www.adaderana.lk/news.ph

https://www.adaderana.lk/news.php?nid=56538
https://www.adaderana.lk/news.php?nid=56228
https://www.adaderana.lk/news.php?nid=55662
https://www.adaderana.lk/news.php?nid=55444
https://www.adaderana.lk/news.php?nid=55348
https://www.adaderana.lk/news.php?nid=55272
https://www.adaderana.lk/news.php?nid=55241
https://www.adaderana.lk/news.php?nid=54870
https://www.adaderana.lk/news.php?nid=54436
https://www.adaderana.lk/news.php?nid=54366
https://www.adaderana.lk/news.php?nid=53980
https://www.adaderana.lk/news.php?nid=53630
https://www.adaderana.lk/news.php?nid=53020
https://www.adaderana.lk/news.php?nid=52773
https://www.adaderana.lk/news.php?nid=51084
https://www.adaderana.lk/news.php?nid=50779
https://www.adaderana.lk/news.php?nid=50770
https://www.adaderana.lk/news.php?nid=49135
https://www.adaderana.lk/news.php?nid=46592
https://www.adaderana.lk/news.php?nid=46517
https://www.adaderana.lk/news.php?nid=46234
https://www.adaderana.lk/news.php?nid=45722
https://www.adaderana.lk/news.ph

https://www.adaderana.lk/news.php?nid=48349
https://www.adaderana.lk/news.php?nid=47688
https://www.adaderana.lk/news.php?nid=47641
https://www.adaderana.lk/news.php?nid=46078
https://www.adaderana.lk/news.php?nid=46063
https://www.adaderana.lk/news.php?nid=44923
https://www.adaderana.lk/news.php?nid=44791
https://www.adaderana.lk/news.php?nid=43160
https://www.adaderana.lk/news.php?nid=41746
https://www.adaderana.lk/news.php?nid=41404
https://www.adaderana.lk/news.php?nid=40489
https://www.adaderana.lk/news.php?nid=39985
https://www.adaderana.lk/news.php?nid=38815
https://www.adaderana.lk/news.php?nid=37925
https://www.adaderana.lk/news.php?nid=37510
https://www.adaderana.lk/news.php?nid=37375
https://www.adaderana.lk/news.php?nid=36541
https://www.adaderana.lk/news.php?nid=34495
https://www.adaderana.lk/news.php?nid=33909
https://www.adaderana.lk/news.php?nid=33110
https://www.adaderana.lk/news.php?nid=33109
https://www.adaderana.lk/news.php?nid=30449
https://www.adaderana.lk/news.ph

In [30]:
# Define a function to remove non-printing characters
def remove_non_printing_chars(s):
    return ''.join(char for char in str(s) if char.isprintable())

In [31]:
# Apply the function to all elements in the DataFrame
df_cleaned = result_df.applymap(remove_non_printing_chars)

In [32]:
print(df_cleaned)

                                             URL  \
0    https://www.adaderana.lk/news.php?nid=29544   
1    https://www.adaderana.lk/news.php?nid=42422   
2    https://www.adaderana.lk/news.php?nid=88904   
3    https://www.adaderana.lk/news.php?nid=86306   
4    https://www.adaderana.lk/news.php?nid=69459   
..                                           ...   
495  https://www.adaderana.lk/news.php?nid=18508   
496  https://www.adaderana.lk/news.php?nid=16702   
497  https://www.adaderana.lk/news.php?nid=16445   
498  https://www.adaderana.lk/news.php?nid=14892   
499  https://www.adaderana.lk/news.php?nid=14272   

                                                 Title  \
0           Sri Lanka female entrepreneurship in focus   
1                   Google fires diversity memo author   
2    President vows to increase women participation...   
3    Youth representation in local govt bodies to b...   
4    Maithripala accused of attempting to shift bla...   
..                         

In [33]:
df_cleaned=df_cleaned.drop_duplicates(subset=['Title'])

In [34]:
df_cleaned.count

<bound method DataFrame.count of                                              URL  \
0    https://www.adaderana.lk/news.php?nid=29544   
1    https://www.adaderana.lk/news.php?nid=42422   
2    https://www.adaderana.lk/news.php?nid=88904   
3    https://www.adaderana.lk/news.php?nid=86306   
4    https://www.adaderana.lk/news.php?nid=69459   
..                                           ...   
495  https://www.adaderana.lk/news.php?nid=18508   
496  https://www.adaderana.lk/news.php?nid=16702   
497  https://www.adaderana.lk/news.php?nid=16445   
498  https://www.adaderana.lk/news.php?nid=14892   
499  https://www.adaderana.lk/news.php?nid=14272   

                                                 Title  \
0           Sri Lanka female entrepreneurship in focus   
1                   Google fires diversity memo author   
2    President vows to increase women participation...   
3    Youth representation in local govt bodies to b...   
4    Maithripala accused of attempting to shift bla.

In [35]:
df_cleaned.to_csv('content/adaderana_20240125.csv', index=False,encoding='utf-8')